In [1]:
import pyomo.environ as pyo

In [2]:
drones = [
    {'name': 'Flexrotor', 'range': 900, 'endurance': 29},
    {'name': 'VBAT', 'range': 300, 'endurance': 10},
    {'name': 'Global Hawk', 'range': 22800, 'endurance': 38},
    {'name': 'Triton', 'range': 15200, 'endurance': 30},
    {'name': 'Reaper', 'range': 1900, 'endurance': 27},
    {'name': 'Gray Eagle', 'range': 6000, 'endurance': 25},
    {'name': 'Shadow', 'range': 110, 'endurance': 6},
    {'name': 'Raven', 'range': 6, 'endurance': 1},
    {'name': 'Puma', 'range': 15, 'endurance': 4},
    {'name': 'Sentinel', 'range': 1668, 'endurance': 6},
    {'name': 'X-47B', 'range': 2400, 'endurance': 4}
]

In [3]:
model = pyo.ConcreteModel()

# define decision vars

model.indices = pyo.RangeSet(len(drones))

model.endurance = pyo.Var(model.indices, initialize = lambda model, i: drones[i-1]['endurance'], within=pyo.NonNegativeIntegers)

model.range = pyo.Var(model.indices, initialize = lambda model, i: drones[i-1]['range'], within=pyo.NonNegativeIntegers)


In [4]:
# set initial params

model.mission_duration = pyo.Param(initialize = 24)
model.min_duration_percent = pyo.Param(initialize = 0.6)
model.min_coverage_percent = pyo.Param(initialize = 0.8)
model.desired_coverage = pyo.Param(initialize = 1000)

In [5]:
# optimize endurance with regards to mission duration
# minimizes the absolute value of distance between desired duration and endurance/2

# def endurance_rule(model):
    # return (sum((model.endurance[i] - model.min_duration_percent * model.mission_duration)**2 for i in model.indices))
min_endurance_diff = min(model.endurance[i] - model.min_duration_percent * model.mission_duration for i in model.indices) #pyo.Objective(rule=endurance_rule, sense=pyo.minimize)
# minimize the square of the differences of each UAV endurance to allowable endurance to ensure all values are positive

#def duration_percentage_constraint_rule(model):
    #return (model.min_duration_percent * model.mission_duration) <= model.mission_duration
    
def non_negative_difference_constraint_rule(model, i):
    return model.endurance[i] - model.min_duration_percent * model.mission_duration >= 0

model.non_negative_difference_constraint = pyo.Constraint(model.indices, rule=non_negative_difference_constraint_rule)
#model.duration_percentage_constraint = pyo.Constraint(rule=duration_percentage_constraint_rule)


PyomoException: Cannot convert non-constant Pyomo expression (endurance[2] - 14.399999999999999  <  endurance[1] - 14.399999999999999) to bool.
This error is usually caused by using a Var, unit, or mutable Param in a
Boolean context such as an "if" statement, or when checking container
membership or equality. For example,
    >>> m.x = Var()
    >>> if m.x >= 1:
    ...     pass
and
    >>> m.y = Var()
    >>> if m.y in [m.x, m.y]:
    ...     pass
would both cause this exception.

In [10]:
# optimize range with regards to mission range
# minimizes the absolute value of distance between desired range and range

# def range_rule(model, best_range):
    # return (sum((model.range[i] - model.min_coverage_percent * model.desired_coverage)**2 for i in model.indices))
min_range_diff = min(model.range[i] - model.min_coverage_percent * model.desired_coverage for i in model.indices) # pyo.Objective(rule=range_rule, sense=pyo.minimize)
# minimize the square of the differences of each UAV range to allowable range to ensure all values are positive

# def coverage_percentage_constraint_rule(model):
    # return model.min_coverage_percent <= 1
    
# model.coverage_percentage_constraint = pyo.Constraint(rule=coverage_percentage_constraint_rule)

In [11]:
# compute weighted sum of obj fxns

weight_1 = 0.5
weight_2 = 0.5

def combined_objective_fxns(model):
    return weight_1 * min_endurance_diff - weight_2 * min_range_diff

model.obj_combined = pyo.Objective(rule = combined_objective_fxns, sense = pyo.minimize)

(type=<class 'pyomo.core.base.objective.ScalarObjective'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.objective.ScalarObjective'>). This is usually indicative of a
modelling error. To avoid this warning, use block.del_component() and
block.add_component().
ERROR: Rule failed when generating expression for Objective obj_combined with
index None: TypeError: unsupported operand type(s) for *: 'float' and
'generator'
ERROR: Constructing component 'obj_combined' from data=None failed:
        TypeError: unsupported operand type(s) for *: 'float' and 'generator'


TypeError: unsupported operand type(s) for *: 'float' and 'generator'

In [8]:
# use glpk as solver

solver = pyo.SolverFactory('ipopt')
solver.solve(model)

model.name="unknown";
    - termination condition: unbounded
    - message from solver: Ipopt 3.14.14\x3a Iterates diverging; problem might
      be unbounded.


{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 0, 'Number of variables': 22, 'Sense': 'unknown'}], 'Solver': [{'Status': 'warning', 'Message': 'Ipopt 3.14.14\\x3a Iterates diverging; problem might be unbounded.', 'Termination condition': 'unbounded', 'Id': 300, 'Error rc': 0, 'Time': 0.08300995826721191}]}